In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pandas_datareader import data as pdr
import datetime

# Escolha do ativo e período
symbol = 'AAPL'
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2025, 7, 25)

# Carrega os dados da fonte 'stooq'
df = pdr.DataReader(symbol, 'stooq', start_date, end_date)[['Close']]

# O stooq retorna na ordem inversa (mais recente primeiro), então invertemos
df = df[::-1]

# Remove valores ausentes
df.dropna(inplace=True)

# Escala os dados
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# Função para criar as sequências de entrada para LSTM
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

X, y = create_sequences(df_scaled)

# Divisão entre treino e teste
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Ajuste para 3D: [samples, time_steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print("Dados carregados com sucesso:", df.shape)

# Treinamento do modelo LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3)],
    verbose=1
)

model.save('model/lstm_model.keras')
from keras.saving import save_model

# Salvar no novo formato oficial do Keras (.keras ZIP-based format)
#save_model(model, 'model/lstm_model.keras')
from keras.saving import save_model

# Salva corretamente como .keras no formato ZIP esperado
save_model(model, 'model/lstm_model.keras')


print("Dados gerados com sucesso.")

Dados carregados com sucesso: (1398, 1)


c:\Users\alexs\OneDrive\Documentos\0 - Projetos - Codificação\Python\Tech_Fase_4\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 109ms/step - loss: 0.0617 - val_loss: 0.0015
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - loss: 0.0029 - val_loss: 0.0014
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - loss: 0.0022 - val_loss: 0.0015
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - loss: 0.0026 - val_loss: 0.0011
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - loss: 0.0021 - val_loss: 0.0010
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0022 - val_loss: 0.0016
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - loss: 0.0020 - val_loss: 0.0015
Epoch 9/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - loss: 0.0019 - val_loss: 0.0012
Dados gerados com sucesso.
